# Build distance maps from antigenic escape assay data

[Greaney et al. 2021](https://doi.org/10.1101/2020.12.31.425021) identified positions in SARS-CoV-2 spike (S) that allowed specific viruses to bind yeast-displayed ACE2 in the presence of antibodies from human polyclonal sera. These assays quantify the degree to which specific mutations enabled antigenic escape in a measure called the `mut_escape`. The resulting data contain the `mut_escape` value for each amino acid mutation from wildtype at each receptor-binding domain (RBD) position in S and specific human serum. The total escape at each site is given by the `total_escape` column.

To understand whether we can use these data to track antigenic evolution of SARS-CoV-2, we developed the following distance maps for use in Nextstrain workflows and eventual visualization in phylogenetic trees:

1. antigenic escape epitope sites: a binary distance map of positions with the strongest immune escape values across all available human sera. This map reflects putative epitope sites akin to the Koel et al. 2013 sites without any consideration of what mutations occur at those sites. This map is most likely to avoid overfitting to a specific serum sample.

2. antigenic escape weighted epitope sites: a floating point distance map for each position in S reflecting the proportion of total antigenic escape provided by that position (total `mut_escape` at that site divided by the total `mut_escape` across all sites and averaged across all available sera). This map avoids using arbitrary thresholds of `mut_escape` values to identify epitope sites by weighting all sites across all sera.

3. antigenic escape by mutation: a site- and mutation-specific floating point distance map reflecting the specific mutational effect of a specific derived state from the corresponding wildtype state. This map provides distances based directly on the absolute measurements of the mutation escape at each position.

Below, we build each map in order of increasing complexity and export the corresponding maps in the format required by the `augur distance` command.

## Imports and configuration

In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
data_url = "https://raw.githubusercontent.com/jbloomlab/SARS-CoV-2-RBD_MAP_HAARVI_sera/main/results/supp_data/human_sera_6m0j_dms-view_data.csv"

In [3]:
antigenic_escape_distance_map = "defaults/distance_maps/greaney_2021_antigenic_escape_ep.json"

In [4]:
weighted_antigenic_escape_distance_map = "defaults/distance_maps/greaney_2021_weighted_antigenic_escape_ep.json"

In [5]:
mutation_antigenic_escape_distance_map = "defaults/distance_maps/greaney_2021_mutation_antigenic_escape.json"

In [6]:
valid_conditions = [
    'subject A (day 120)',
    'subject B (day 113)',
    'subject C (day 104)',
    'subject D (day 76)',
    'subject E (day 104)',
    'subject F (day 115)',
    'subject G (day 94)',
    'subject H (day 152)',
    'subject I (day 102)',
    'subject J (day 121)',
    'subject K (day 103)'
]

## Load data

In [7]:
df = pd.read_csv(data_url)

In [8]:
df.head()

,condition,site,label_site,wildtype,mutation,protein_chain,protein_site,mut_escape color ACE2 bind,site_total escape,site_max escape,color_for_mutation,mut_escape color RBD expr,mut_escape color gray,mut_escape color func group
0,subject H (day 152),1,331,N,A,E,331,0.002020,0.04926,0.007632,#692505,NaN,NaN,NaN
1,subject H (day 152),1,331,N,D,E,331,0.005616,0.04926,0.007632,#662505,NaN,NaN,NaN
2,subject H (day 152),1,331,N,E,E,331,0.002535,0.04926,0.007632,#662505,NaN,NaN,NaN
3,subject H (day 152),1,331,N,F,E,331,0.003032,0.04926,0.007632,#722805,NaN,NaN,NaN
4,subject H (day 152),1,331,N,G,E,331,0.003113,0.04926,0.007632,#6a2605,NaN,NaN,NaN


In [9]:
df["condition"].value_counts().shape

(23,)

In [10]:
df = df[df["condition"].isin(valid_conditions)].copy()

In [11]:
def get_subject_from_condition(condition):
    return condition.split("(")[0].strip()

In [12]:
def get_day_from_condition(condition):
    return int(condition.split(" ")[-1].replace(")", ""))

In [13]:
df["subject"] = df["condition"].apply(get_subject_from_condition)

In [14]:
df["day"] = df["condition"].apply(get_day_from_condition)

In [15]:
df["subject"].value_counts()

subject G    7792
subject H    7784
subject D    7780
subject K    7780
subject E    7780
subject C    7780
subject A    7772
subject B    7772
subject J    7772
subject I    7772
subject F    7772
Name: subject, dtype: int64

In [16]:
df["day"].value_counts()

104    15560
94      7792
152     7784
103     7780
76      7780
121     7772
120     7772
115     7772
113     7772
102     7772
Name: day, dtype: int64

In [17]:
sorted(df.sort_values(
    ["subject", "day"],
    ascending=False
).groupby(
    ["subject"],
    sort=False
).first()["condition"].values)

['subject A (day 120)',
 'subject B (day 113)',
 'subject C (day 104)',
 'subject D (day 76)',
 'subject E (day 104)',
 'subject F (day 115)',
 'subject G (day 94)',
 'subject H (day 152)',
 'subject I (day 102)',
 'subject J (day 121)',
 'subject K (day 103)']

In [18]:
df.head()

,condition,site,label_site,wildtype,mutation,protein_chain,protein_site,mut_escape color ACE2 bind,site_total escape,site_max escape,color_for_mutation,mut_escape color RBD expr,mut_escape color gray,mut_escape color func group,subject,day
0,subject H (day 152),1,331,N,A,E,331,0.002020,0.04926,0.007632,#692505,NaN,NaN,NaN,subject H,152
1,subject H (day 152),1,331,N,D,E,331,0.005616,0.04926,0.007632,#662505,NaN,NaN,NaN,subject H,152
2,subject H (day 152),1,331,N,E,E,331,0.002535,0.04926,0.007632,#662505,NaN,NaN,NaN,subject H,152
3,subject H (day 152),1,331,N,F,E,331,0.003032,0.04926,0.007632,#722805,NaN,NaN,NaN,subject H,152
4,subject H (day 152),1,331,N,G,E,331,0.003113,0.04926,0.007632,#6a2605,NaN,NaN,NaN,subject H,152


In [19]:
df["label_site"].drop_duplicates().sort_values().values

array([331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 344,
       345, 346, 347, 348, 349, 351, 352, 353, 354, 356, 357, 358, 359,
       360, 361, 362, 363, 365, 366, 367, 368, 369, 370, 371, 372, 373,
       374, 375, 376, 377, 378, 380, 381, 382, 383, 384, 385, 386, 387,
       388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 399, 401, 402,
       403, 404, 405, 406, 407, 408, 409, 410, 411, 413, 414, 415, 417,
       418, 419, 420, 421, 424, 425, 427, 428, 429, 430, 433, 434, 435,
       437, 438, 439, 440, 441, 443, 444, 445, 446, 447, 448, 449, 450,
       451, 452, 453, 455, 456, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479,
       481, 482, 483, 484, 485, 486, 487, 489, 490, 492, 493, 494, 496,
       498, 499, 500, 501, 503, 504, 505, 506, 507, 508, 510, 512, 513,
       514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526,
       527, 528, 529, 530, 531])

In [20]:
# All sites in this analysis are in the same gene, spike (S).
df["gene"] = "S"

In [21]:
df["gene_site"] = df["label_site"]

In [22]:
df.head()

,condition,site,label_site,wildtype,mutation,protein_chain,protein_site,mut_escape color ACE2 bind,site_total escape,site_max escape,color_for_mutation,mut_escape color RBD expr,mut_escape color gray,mut_escape color func group,subject,day,gene,gene_site
0,subject H (day 152),1,331,N,A,E,331,0.002020,0.04926,0.007632,#692505,NaN,NaN,NaN,subject H,152,S,331
1,subject H (day 152),1,331,N,D,E,331,0.005616,0.04926,0.007632,#662505,NaN,NaN,NaN,subject H,152,S,331
2,subject H (day 152),1,331,N,E,E,331,0.002535,0.04926,0.007632,#662505,NaN,NaN,NaN,subject H,152,S,331
3,subject H (day 152),1,331,N,F,E,331,0.003032,0.04926,0.007632,#722805,NaN,NaN,NaN,subject H,152,S,331
4,subject H (day 152),1,331,N,G,E,331,0.003113,0.04926,0.007632,#6a2605,NaN,NaN,NaN,subject H,152,S,331


In [23]:
mutations_per_site = df.loc[:, ["label_site", "wildtype", "mutation"]].drop_duplicates().copy()

In [24]:
mutations_per_site.query("label_site == 331")

,label_site,wildtype,mutation
0,331,N,A
1,331,N,D
2,331,N,E
3,331,N,F
4,331,N,G
5,331,N,H
6,331,N,I
7,331,N,K
8,331,N,L
9,331,N,M


In [25]:
mutations_per_site["mutation"].drop_duplicates().shape

(20,)

In [26]:
df["condition"].value_counts()

subject G (day 94)     7792
subject H (day 152)    7784
subject K (day 103)    7780
subject D (day 76)     7780
subject C (day 104)    7780
subject E (day 104)    7780
subject F (day 115)    7772
subject B (day 113)    7772
subject A (day 120)    7772
subject I (day 102)    7772
subject J (day 121)    7772
Name: condition, dtype: int64

In [27]:
df["subject"].value_counts()

subject G    7792
subject H    7784
subject D    7780
subject K    7780
subject E    7780
subject C    7780
subject A    7772
subject B    7772
subject J    7772
subject I    7772
subject F    7772
Name: subject, dtype: int64

## Identify antigenic escape epitope sites

Calculate site-specific total mut_escape values, plot their distributions per serum, and identify those positions with values greater than 4 standard deviations from the mean by serum.

In [28]:
escape_column = "site_total escape"

In [29]:
site_columns = [
    "condition",
    "site",
    "gene",
    "gene_site",
    escape_column
]

In [30]:
site_specific_df = df.loc[:, site_columns].drop_duplicates().sort_values(["condition", "site"]).reset_index().drop(columns=["index"])

In [31]:
site_specific_df.head()

,condition,site,gene,gene_site,site_total escape
0,subject A (day 120),1,S,331,0.1205
1,subject A (day 120),2,S,332,0.6332
2,subject A (day 120),3,S,333,0.1667
3,subject A (day 120),4,S,334,0.5097
4,subject A (day 120),5,S,335,0.6866


In [32]:
site_specific_df.shape

(1914, 5)

In [33]:
condition_specific_distributions = site_specific_df.groupby("condition").aggregate({escape_column: ["mean", "std"]})

In [34]:
condition_specific_distributions["upper_threshold"] = (
    condition_specific_distributions[escape_column, "mean"] + 4 * condition_specific_distributions[escape_column, "std"]
)

In [35]:
upper_thresholds = condition_specific_distributions.reset_index().loc[:, ["condition", "upper_threshold"]]

In [36]:
upper_thresholds.columns = upper_thresholds.columns.droplevel(level=1)

In [37]:
upper_thresholds

,condition,upper_threshold
0,subject A (day 120),2.785092
1,subject B (day 113),2.134808
2,subject C (day 104),0.305421
3,subject D (day 76),0.047514
4,subject E (day 104),3.191500
5,subject F (day 115),3.365717
6,subject G (day 94),2.024142
7,subject H (day 152),2.224005
8,subject I (day 102),4.384038
9,subject J (day 121),2.049182


In [38]:
site_specific_df = site_specific_df.merge(
    upper_thresholds,
    on="condition"
)

In [39]:
site_specific_df[site_specific_df[escape_column] > site_specific_df["upper_threshold"]]

,condition,site,gene,gene_site,site_total escape,upper_threshold
108,subject A (day 120),126,S,456,3.7920,2.785092
133,subject A (day 120),154,S,484,3.0420,2.785092
282,subject B (day 113),126,S,456,3.5100,2.134808
307,subject B (day 113),154,S,484,3.0190,2.134808
456,subject C (day 104),126,S,456,0.6150,0.305421
481,subject C (day 104),154,S,484,0.7445,0.305421
630,subject D (day 76),126,S,456,0.1029,0.047514
804,subject E (day 104),126,S,456,3.7300,3.191500
818,subject E (day 104),142,S,472,3.6470,3.191500
829,subject E (day 104),154,S,484,8.5360,3.191500


In [40]:
antigenic_epitope_sites = site_specific_df.loc[
    site_specific_df[escape_column] > site_specific_df["upper_threshold"],
    ["gene", "gene_site"]
].drop_duplicates().sort_values(["gene", "gene_site"])

In [41]:
antigenic_epitope_sites

,gene,gene_site
1318,S,447
972,S,449
1323,S,452
108,S,456
818,S,472
133,S,484
1527,S,486
1530,S,490


In [42]:
antigenic_epitope_sites.shape

(8, 2)

In [43]:
antigenic_escape_map = {
    "name": "antigenic_escape_epitope_sites",
    "description": f"Sites with greater than 4 stddev site-level escape per serum across {len(valid_conditions)} human sera from Greaney et al. 2021",
    "default": 0,
    "map": {}
}

In [44]:
for record in antigenic_epitope_sites.to_dict("records"):
    if record["gene"] not in antigenic_escape_map["map"]:
        antigenic_escape_map["map"][record["gene"]] = {}
        
    antigenic_escape_map["map"][record["gene"]][str(record["gene_site"])] = 1

In [45]:
antigenic_escape_map

{'name': 'antigenic_escape_epitope_sites',
 'description': 'Sites with greater than 4 stddev site-level escape per serum across 11 human sera from Greaney et al. 2021',
 'default': 0,
 'map': {'S': {'447': 1,
   '449': 1,
   '452': 1,
   '456': 1,
   '472': 1,
   '484': 1,
   '486': 1,
   '490': 1}}}

In [46]:
with open(antigenic_escape_distance_map, "w") as oh:
    json.dump(antigenic_escape_map, oh, indent=4, sort_keys=True)

In [47]:
!pwd

/Users/jlhudd/projects/nextstrain/ncov


## Identify weighted antigenic escape epitope sites

Instead of using an arbitrary threshold (e.g., 4 std dev from the mean), calculate the average proportion of the total escape value at each position in S across all sera. The resulting distance map weights each position in S by its relative contribution to antigenic escape.

In [48]:
total_escape = site_specific_df.groupby("condition").aggregate({escape_column: "sum"}).reset_index()

In [49]:
total_escape = total_escape.rename(columns={escape_column: "total_escape"})

In [50]:
total_escape

,condition,total_escape
0,subject A (day 120),92.674160
1,subject B (day 113),60.538330
2,subject C (day 104),2.690213
3,subject D (day 76),1.634826
4,subject E (day 104),22.716179
5,subject F (day 115),62.018851
6,subject G (day 94),95.918280
7,subject H (day 152),42.703905
8,subject I (day 102),66.123663
9,subject J (day 121),86.877970


In [51]:
site_specific_df = site_specific_df.merge(
    total_escape,
    on="condition"
)

In [52]:
site_specific_df.head()

,condition,site,gene,gene_site,site_total escape,upper_threshold,total_escape
0,subject A (day 120),1,S,331,0.1205,2.785092,92.67416
1,subject A (day 120),2,S,332,0.6332,2.785092,92.67416
2,subject A (day 120),3,S,333,0.1667,2.785092,92.67416
3,subject A (day 120),4,S,334,0.5097,2.785092,92.67416
4,subject A (day 120),5,S,335,0.6866,2.785092,92.67416


Sum total escape values per site and condition and calculate the proportion of these totals per site. This approach averages the antigenic escape effect per site across all conditions (sera).

In [53]:
total_site_specific_escape = site_specific_df.groupby(["gene", "gene_site"]).aggregate({
    escape_column: "sum",
    "total_escape": "sum"
}).reset_index()

In [54]:
total_site_specific_escape["proportion_of_escape"] = (
    total_site_specific_escape[escape_column] / total_site_specific_escape["total_escape"]
)

In [55]:
total_site_specific_escape

,gene,gene_site,site_total escape,total_escape,proportion_of_escape
0,S,331,1.478387,540.261779,0.002736
1,S,332,3.153901,540.261779,0.005838
2,S,333,1.640904,540.261779,0.003037
3,S,334,2.693905,540.261779,0.004986
4,S,335,3.239830,540.261779,0.005997
...,...,...,...,...,...
169,S,527,2.846798,540.261779,0.005269
170,S,528,3.524410,540.261779,0.006524
171,S,529,3.942250,540.261779,0.007297
172,S,530,3.148190,540.261779,0.005827


In [56]:
total_site_specific_escape.query("gene == 'S' & gene_site == 484")

,gene,gene_site,site_total escape,total_escape,proportion_of_escape
133,S,484,33.45514,540.261779,0.061924


In [57]:
weighted_antigenic_escape_map = {
    "name": "antigenic_escape_weighted_epitope_sites",
    "description": "Average proportion of site-level total escape per site across four human sera from Greaney et al. 2021",
    "default": 0.0,
    "map": {}
}

In [58]:
for record in total_site_specific_escape.to_dict("records"):
    if record["gene"] not in weighted_antigenic_escape_map["map"]:
        weighted_antigenic_escape_map["map"][record["gene"]] = {}
        
    weighted_antigenic_escape_map["map"][record["gene"]][str(record["gene_site"])] = record["proportion_of_escape"]

In [59]:
weighted_antigenic_escape_map

{'name': 'antigenic_escape_weighted_epitope_sites',
 'description': 'Average proportion of site-level total escape per site across four human sera from Greaney et al. 2021',
 'default': 0.0,
 'map': {'S': {'331': 0.0027364271507114802,
   '332': 0.005837727419854265,
   '333': 0.003037238732017442,
   '334': 0.004986295728680927,
   '335': 0.0059967780937532425,
   '336': 0.0011751290298753962,
   '337': 0.004510424567535593,
   '338': 0.0015981696908446933,
   '339': 0.005228752265752545,
   '340': 0.0026803728429881666,
   '341': 0.0008037307413537133,
   '342': 0.0007009435330426895,
   '344': 0.0015493600192470253,
   '345': 0.0014853558617128663,
   '346': 0.0065186454755736655,
   '347': 0.000519251797939699,
   '348': 0.0033662496059586144,
   '349': 0.0007261368754816679,
   '351': 0.0011045238131141325,
   '352': 0.0030244301265014823,
   '353': 0.00029253244667990197,
   '354': 0.0030305234688943355,
   '356': 0.004199971363956128,
   '357': 0.005834634474793981,
   '358': 0.

In [60]:
with open(weighted_antigenic_escape_distance_map, "w") as oh:
    json.dump(weighted_antigenic_escape_map, oh, indent=4, sort_keys=True)

## Calculate antigenic escape effects for specific mutations

Follow the approach from Lee et al. 2018 where mutational effects were estimated by the log ratio of DMS preference values of the derived and ancestral amino acids at a given site.

Build a distance map for each mutation from one amino acid to another using this equation. For any mutations that are not defined (e.g., mutations to wildtype at a site), the preference is zero.

In [61]:
escape_by_mutation = df.query("condition == 'subject H (day 152)'").loc[
    :, ["condition", "label_site", "wildtype", "mutation", "mut_escape color ACE2 bind"]
].dropna().rename(
    columns={"mut_escape color ACE2 bind": "mut_escape"}
).groupby([
    "label_site",
    "wildtype",
    "mutation"
])["mut_escape"].mean().reset_index()

In [62]:
escape_by_mutation

,label_site,wildtype,mutation,mut_escape
0,331,N,A,0.002020
1,331,N,D,0.005616
2,331,N,E,0.002535
3,331,N,F,0.003032
4,331,N,G,0.003113
...,...,...,...,...
1941,531,T,R,0.003578
1942,531,T,S,0.008192
1943,531,T,V,0.002950
1944,531,T,W,0.002530


In [63]:
gene = "S"
mutation_antigenic_escape_map = {
    "name": "antigenic_escape_mutation_effect",
    "default": 0.0,
    "map": {
        gene: {}
    }
}

for index, row in escape_by_mutation.iterrows():
    position = str(row["label_site"])
            
    if position not in mutation_antigenic_escape_map["map"][gene]:
        mutation_antigenic_escape_map["map"][gene][position] = []
    
    mutation_antigenic_escape_map["map"][gene][position].append({
        "from": row["wildtype"],
        "to": row["mutation"],
        "weight": row["mut_escape"]
    })

In [64]:
mutation_antigenic_escape_map["map"]["S"]["484"]

[{'from': 'E', 'to': 'A', 'weight': 0.2072},
 {'from': 'E', 'to': 'C', 'weight': 0.1541},
 {'from': 'E', 'to': 'D', 'weight': 0.01901},
 {'from': 'E', 'to': 'F', 'weight': 0.2066},
 {'from': 'E', 'to': 'G', 'weight': 0.05396},
 {'from': 'E', 'to': 'H', 'weight': 0.2303},
 {'from': 'E', 'to': 'I', 'weight': 0.2216},
 {'from': 'E', 'to': 'K', 'weight': 0.14300000000000002},
 {'from': 'E', 'to': 'L', 'weight': 0.1879},
 {'from': 'E', 'to': 'M', 'weight': 0.1632},
 {'from': 'E', 'to': 'N', 'weight': 0.04948},
 {'from': 'E', 'to': 'P', 'weight': 0.2348},
 {'from': 'E', 'to': 'Q', 'weight': 0.1674},
 {'from': 'E', 'to': 'R', 'weight': 0.1387},
 {'from': 'E', 'to': 'S', 'weight': 0.1491},
 {'from': 'E', 'to': 'T', 'weight': 0.1291},
 {'from': 'E', 'to': 'V', 'weight': 0.21},
 {'from': 'E', 'to': 'W', 'weight': 0.1513},
 {'from': 'E', 'to': 'Y', 'weight': 0.1815}]

In [65]:
with open(mutation_antigenic_escape_distance_map, "w") as oh:
    json.dump(mutation_antigenic_escape_map, oh, indent=4, sort_keys=True)